In [1]:
import pandas as pd
import os
import numpy as np
from multiprocessing import Pool
import time

import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")


In [2]:
future2stock = {'JBF': 3443, 'QWF': 2388, 'HCF': 2498, 'DBF': 2610, 'EHF': 1319, 'IPF': 3035, 'IIF': 3006, 'QXF': 2615, 'PEF': 5425, 'NAF': 3105}
future_path = r'processedData_2023\\futures'
stock_path = r'processedData_2023\\stocks'
future_files = os.listdir(future_path)
stock_files = os.listdir(stock_path)

# Get all the future tickers
future_data = {}
stock_data = {}
for future_ticker, stock_ticker in future2stock.items():
    future_data[future_ticker] = pd.DataFrame()
    stock_data[stock_ticker] = pd.DataFrame()
    for file in future_files:
        if file.startswith(future_ticker):
            tmp = pd.read_csv(os.path.join(future_path, file), compression='gzip')    
            tmp = tmp[tmp['askPrice1'] > 0]
            tmp = tmp[tmp['bidPrice1'] > 0]
            tmp = tmp[tmp['askPrice1'] > tmp['bidPrice1']]
            future_data[future_ticker] = pd.concat([future_data[future_ticker], tmp])
            
    for file in stock_files:
        if file.startswith(str(stock_ticker)):
            tmp = pd.read_csv(os.path.join(stock_path, file), compression='gzip')
            tmp = tmp[tmp['SP1'] > 0]
            tmp = tmp[tmp['BP1'] > 0]
            tmp = tmp[tmp['SP1'] > tmp['BP1']]
            stock_data[stock_ticker] = pd.concat([stock_data[stock_ticker], tmp])

In [3]:
# Select common dates and indexing
for future_ticker, futureData in future_data.items():
    stockData = stock_data[future2stock[future_ticker]]
    stockData_dates = np.unique(stockData.date)
    stoD=pd.to_datetime(stockData_dates, format="%Y-%m-%d")
    qqqq=stoD.year*10000+stoD.month*100+stoD.day
    
    indexData_dates = np.unique(futureData.date)
    indD=pd.to_datetime(indexData_dates, format="%Y-%m-%d")
    pppp=indD.year*10000+indD.month*100+indD.day
    
    commonDays=pd.to_datetime(pppp.intersection(qqqq),format="%Y%m%d")
    
    d_futures=pd.to_datetime(futureData.date,format="%Y-%m-%d")
    futureData.date=d_futures
    futureData=futureData[futureData.date.isin(commonDays)]

    d_stock=pd.to_datetime(stockData.date,format="%Y-%m-%d")
    stockData.date=d_stock
    stockData=stockData[stockData.date.isin(commonDays)]

    stockData_DateTime = pd.to_datetime(stockData.date.astype(str) + ' ' + stockData.time.astype(str), format="%Y-%m-%d %H%M%S%f")
    futuresData_DateTime = pd.to_datetime(futureData.date.astype(str) + ' ' + futureData.time.astype(str), format="%Y-%m-%d %H%M%S%f")

    stockData.index = stockData_DateTime
    stockData = stockData[~stockData.index.duplicated(keep='last')]

    futureData.index = futuresData_DateTime
    futureData = futureData[~futureData.index.duplicated(keep='last')]
    
    stockData = stockData.sort_index()
    futureData = futureData.sort_index()
    
    stock_data[future2stock[future_ticker]] = stockData
    future_data[future_ticker] = futureData
    
# Downsampling
freq = '10s'
for future_ticker, futureData in future_data.items():
    stockData = stock_data[future2stock[future_ticker]]
    stockData = stockData.groupby(stockData.index.date).apply(lambda x: x.resample(freq, closed='left', label='right').last()).reset_index(level=0, drop=True)
    futureData = futureData.groupby(futureData.index.date).apply(lambda x: x.resample(freq, closed='left', label='right').last()).reset_index(level=0, drop=True)
    stockData.fillna(method='ffill', inplace=True)
    futureData.fillna(method='ffill', inplace=True)
    # get the common index
    comon_index = stockData.index.intersection(futureData.index)
    stock_data[future2stock[future_ticker]] = stockData.loc[comon_index]
    future_data[future_ticker] = futureData.loc[comon_index]

In [4]:
# Order Slope
def cal_slope(df):
    # Consider future and stock difference
    if 'bidSize1' in df.columns:
        bidSizes = [f'bidSize{i}' for i in range(1, 6)]
        bidPrices = [f'bidPrice{i}' for i in range(1, 6)]
        askSizes = [f'askSize{i}' for i in range(1, 6)]
        askPrices = [f'askPrice{i}' for i in range(1, 6)]
    else:
        bidSizes = [f'BV{i}' for i in range(1, 6)]
        bidPrices = [f'BP{i}' for i in range(1, 6)]
        askSizes = [f'SV{i}' for i in range(1, 6)]
        askPrices = [f'SP{i}' for i in range(1, 6)]

    df_bid = df[bidSizes + bidPrices].copy()
    df_ask = df[askSizes + askPrices].copy()
    df_bid.loc[:, bidPrices] = df_bid[bidPrices] / df[bidPrices[0]].values.reshape(-1, 1)
    df_ask.loc[:, askPrices] = df_ask[askPrices] / df[askPrices[0]].values.reshape(-1, 1)
    
    bid_data = df_bid.values
    ask_data = df_ask.values

    cum_bid_sizes = np.cumsum(bid_data[:, :5], axis=1) / bid_data[:, :5].sum(axis=1, keepdims=True)
    cum_ask_sizes = np.cumsum(ask_data[:, :5], axis=1) / ask_data[:, :5].sum(axis=1, keepdims=True)

    bid_price = bid_data[:, 5:]
    ask_price = ask_data[:, 5:]

    X_bid = cum_bid_sizes - cum_bid_sizes.mean(axis=1, keepdims=True)
    Y_bid = bid_price - bid_price.mean(axis=1, keepdims=True)
    slope_b = (X_bid * Y_bid).sum(axis=1) / ((X_bid ** 2).sum(axis=1) + 1e-10)

    X_ask = cum_ask_sizes - cum_ask_sizes.mean(axis=1, keepdims=True)
    Y_ask = ask_price - ask_price.mean(axis=1, keepdims=True)
    slope_a = (X_ask * Y_ask).sum(axis=1) / ((X_ask ** 2).sum(axis=1) + 1e-10)

    return -slope_b, slope_a

In [6]:
# Add features and labels' names to list
df_all = {}
basicCols = ['date', 'time', 'sAskPrice1','sBidPrice1','sMidQ', 'fAskPrice1','fBidPrice1', 'fMidQ', 'spreadRatio']
labelCols = []
featureCols = []

for i in range(1, 11):
    labelCols.extend(['Y_M_{}'.format(str(i))])
    
    featureCols.extend(['fLaggingRtn_{}'.format(str(i))])
    featureCols.extend(['spreadRatio_{}'.format(str(i))])
    featureCols.extend(['volumeImbalanceRatio_{}'.format(str(i))])
    featureCols.extend(['slope_b_{}'.format(str(i))])
    featureCols.extend(['slope_a_{}'.format(str(i))])
    featureCols.extend(['slope_ab_{}'.format(str(i))])
    featureCols.extend(['sLaggingRtn_{}'.format(str(i))])
    featureCols.extend(['stockSpreadRatio_{}'.format(str(i))])
    featureCols.extend(['stockVolumeImbalanceRatio_{}'.format(str(i))])
    featureCols.extend(['stockSlope_b_{}'.format(str(i))])
    featureCols.extend(['stockSlope_a_{}'.format(str(i))])
    featureCols.extend(['stockSlope_ab_{}'.format(str(i))])
    
    for j in range(1, 6):
        featureCols.extend(['fAskSize{}_{}'.format(str(j), str(i))])
        featureCols.extend(['fBidSize{}_{}'.format(str(j), str(i))])
        featureCols.extend(['sAskSize{}_{}'.format(str(j), str(i))])
        featureCols.extend(['sBidSize{}_{}'.format(str(j), str(i))])

# Generate features and labels
for future_ticker, futureData in future_data.items():
    stockData = stock_data[future2stock[future_ticker]]
    unique_dates = np.unique(stockData['date'])
    dfs = []
    for date in unique_dates:
        stockData_date = stockData[stockData['date'] == date]
        futureData_date = futureData[futureData['date'] == date]

        # Continue to next iteration if stockData_date or futureData_date is empty
        if stockData_date.empty or futureData_date.empty:
            continue

        df = pd.DataFrame(index=stockData_date.index, columns=basicCols+labelCols+featureCols)
        df['date'] = stockData_date['date']
        df['time'] = stockData_date['time']   
             
        # Normalize the size
        fAskSizeMax = futureData_date[['askSize1', 'askSize2', 'askSize3', 'askSize4', 'askSize5']].max(axis=1)
        fBidSizeMax = futureData_date[['bidSize1', 'bidSize2', 'bidSize3', 'bidSize4', 'bidSize5']].max(axis=1)
        sAskSizeMax = stockData_date[['SV1', 'SV2', 'SV3', 'SV4', 'SV5']].max(axis=1)
        sBidSizeMax = stockData_date[['BV1', 'BV2', 'BV3', 'BV4', 'BV5']].max(axis=1)
        
        for i in range(1, 6):
            df['fAskPrice{}'.format(str(i))] = futureData_date['askPrice{}'.format(str(i))]
            df['fBidPrice{}'.format(str(i))] = futureData_date['bidPrice{}'.format(str(i))]
            df['fAskSize{}'.format(str(i))] = futureData_date['askSize{}'.format(str(i))] / fAskSizeMax
            df['fBidSize{}'.format(str(i))] = futureData_date['bidSize{}'.format(str(i))] / fBidSizeMax
            
            df['sAskPrice{}'.format(str(i))] = stockData_date['SP{}'.format(str(i))]
            df['sBidPrice{}'.format(str(i))] = stockData_date['BP{}'.format(str(i))]
            df['sAskSize{}'.format(str(i))] = stockData_date['SV{}'.format(str(i))] / sAskSizeMax
            df['sBidSize{}'.format(str(i))] = stockData_date['BV{}'.format(str(i))] / sBidSizeMax
        
        df['fMidQ'] = (df['fAskPrice1'] + df['fBidPrice1']) / 2
        df['slope_b'], df['slope_a'] = cal_slope(futureData_date)
        df['slope_ab'] = df['slope_a'] - df['slope_b']
        
        # Order Imbalance Ratio
        ask = np.array([df['fAskPrice{}'.format(str(i))] * df['fAskSize{}'.format(str(i))] * (1 - (i - 1) / 5) for i in range(1, 6)]).sum(axis=0)
        bid = np.array([df['fBidPrice{}'.format(str(i))] * df['fBidSize{}'.format(str(i))] * (1 - (i - 1) / 5) for i in range(1, 6)]).sum(axis=0)
        df['spreadRatio'] = (ask - bid) / (ask + bid)

        delta_size_bid = np.where(df['fBidPrice1'] < df['fBidPrice1'].shift(1), 0, np.where(df['fBidPrice1'] == df['fBidPrice1'].shift(1), df['fBidSize1'] - df['fBidSize1'].shift(1), df['fBidSize1']))
        delta_size_ask = np.where(df['fAskPrice1'].shift(1), 0, np.where(df['fAskPrice1'] == df['fAskPrice1'].shift(1), df['fAskSize1'] - df['fAskSize1'].shift(1), df['fAskSize1']))
        df['fOrderImbalance'] = (delta_size_bid - delta_size_ask) / (delta_size_bid + delta_size_ask)

        for i in range(1, 11):
            df['fLaggingRtn_{}'.format(str(i))] = np.log(df['fMidQ']) - np.log(df['fMidQ'].shift(i))
            df['spreadRatio_{}'.format(str(i))] = df['spreadRatio'].shift(i)
            df['volumeImbalanceRatio_{}'.format(str(i))] = df['fOrderImbalance'].shift(i)
            df['slope_b_{}'.format(str(i))] = df['slope_b'].shift(i)
            df['slope_a_{}'.format(str(i))] = df['slope_a'].shift(i)
            df['slope_ab_{}'.format(str(i))] = df['slope_ab'].shift(i)
            
            for j in range(1, 6):
                df['fAskSize{}_{}'.format(str(j), str(i))] = df['fAskSize{}'.format(str(j))].shift(i)
                df['fBidSize{}_{}'.format(str(j), str(i))] = df['fBidSize{}'.format(str(j))].shift(i)
                df['sAskSize{}_{}'.format(str(j), str(i))] = df['sAskSize{}'.format(str(j))].shift(i)
                df['sBidSize{}_{}'.format(str(j), str(i))] = df['sBidSize{}'.format(str(j))].shift(i)

        # Add stock data
        df['sMidQ'] = (stockData_date['SP1'] + stockData_date['BP1'])/2
        df['sAskPrice1'] = stockData_date['SP1']
        df['sBidPrice1'] = stockData_date['BP1']
        df['sMidQ'] = (stockData_date['SP1'] + stockData_date['BP1'])/2
        df['stockSlope_b'], df['stockSlope_a'] = cal_slope(stockData_date)
        df['stockSlope_ab'] = df['stockSlope_a'] - df['stockSlope_b']
        
        ask = np.array([df['sAskPrice{}'.format(str(i))] * df['sAskSize{}'.format(str(i))] * (1 - (i - 1) / 5) for i in range(1, 6)]).sum(axis=0)
        bid = np.array([df['sBidPrice{}'.format(str(i))] * df['sBidSize{}'.format(str(i))] * (1 - (i - 1) / 5) for i in range(1, 6)]).sum(axis=0)
        df['stockSpreadRatio'] = (ask - bid) / (ask + bid)

        delta_size_bid = np.where(df['sBidPrice1'] < df['sBidPrice1'].shift(1), 0, np.where(df['sBidPrice1'] == df['sBidPrice1'].shift(1), df['sBidSize1'] - df['sBidSize1'].shift(1), df['sBidSize1']))
        delta_size_ask = np.where(df['sAskPrice1'] > df['sAskPrice1'].shift(1), 0, np.where(df['sAskPrice1'] == df['sAskPrice1'].shift(1), df['sAskSize1'] - df['sAskSize1'].shift(1), df['sAskSize1']))
        df['stockOrderImbalance'] = (delta_size_bid - delta_size_ask) / (delta_size_bid + delta_size_ask)

        for i in range(1, 11):
            df['Y_M_{}'.format(str(i))] = np.log(df['sMidQ'].shift(-i)) - np.log(df['sMidQ'])
            
            df['sLaggingRtn_{}'.format(str(i))] = np.log(df['sMidQ']) - np.log(df['sMidQ'].shift(i))
            df['stockSpreadRatio_{}'.format(str(i))] = df['stockSpreadRatio'].shift(i)
            df['stockVolumeImbalanceRatio_{}'.format(str(i))] = df['stockOrderImbalance'].shift(i)
            df['stockSlope_b_{}'.format(str(i))] = df['stockSlope_b'].shift(i)
            df['stockSlope_a_{}'.format(str(i))] = df['stockSlope_a'].shift(i)
            df['stockSlope_ab_{}'.format(str(i))] = df['stockSlope_ab'].shift(i)
            
            for j in range(1, 6):
                df['sAskSize{}_{}'.format(str(j), str(i))] = df['sAskSize{}'.format(str(j))].shift(i)
                df['sBidSize{}_{}'.format(str(j), str(i))] = df['sBidSize{}'.format(str(j))].shift(i)
        dfs.append(df)
    # convert inf to nan to 0
    df_all[future_ticker] = pd.concat(dfs, ignore_index=True).replace([np.inf, -np.inf], np.nan).fillna(0)

In [15]:
import optuna
from optuna.integration import LightGBMPruningCallback


def objective(trial, x_train, y_train, measure, lookforward_win):
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': 500,
        'learning_rate': 0.05,
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7]), 
        'num_leaves': trial.suggest_categorical('num_leaves', [7, 31, 63, 127]),
        'subsample': trial.suggest_categorical('subsample', [0.8]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.8]),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    }

    model = lgb.LGBMRegressor(**param)
    tscv = TimeSeriesSplit(n_splits=5)
    score = 0
    for train_index, val_index in tscv.split(x_train):
        x_train_cv, x_val_cv = x_train.iloc[train_index], x_train.iloc[val_index]
        y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]
        model.fit(x_train_cv, y_train_cv, eval_set=[(x_val_cv, y_val_cv)], early_stopping_rounds=50, verbose=False, callbacks=[LightGBMPruningCallback(trial, 'rmse')])
        y_pred = model.predict(x_val_cv)
        if measure == 'rmse':
            error = mean_squared_error(y_val_cv, y_pred)
            score += error
        elif measure == 'cum_ret':
            error = -backtest(y_val_cv, y_pred, lookforward_win)
            score += error
    return score / 5


def backtest(y_val_cv, pred_series, lookforward_win):
    val_df = y_val_cv.copy()
    val_df = pd.DataFrame(val_df)
    val_df['pred'] = pred_series

    # long if pred > 0, short if pred < 0, record the return
    val_df['direction'] = np.where(val_df['pred'] > 0, 1, np.where(val_df['pred'] < 0, -1, 0))
    val_df['return'] = val_df['Y_M_{}'.format(str(lookforward_win))] * val_df['direction'] / lookforward_win
    val_df['return'] = val_df['return'].shift(lookforward_win)
    val_df['return'] = val_df['return'].fillna(0)

    # calculate the cumulative return
    val_df['cum_return'] = val_df['return'].cumsum() + 1
    cum_return = val_df['cum_return'].iloc[-1]
    print(cum_return)

    # std of return
    val_df['std'] = val_df['return'].std()

    return cum_return


def modelConstructionAndForecasting(numOfPastDays:int, numOfForwardDays:int, data:pd.DataFrame, numOfProcesses:int, featureColumns:list, tickers:str, measure:str):
    date=data.date
    date_index=np.unique(date)
    date_num=date_index.size
    p=numOfPastDays
    
    #Output data
    columns1 = ['date', 'label', 'isR2', 'oosR2']
    outputData = pd.DataFrame(columns=columns1)
    
    tscv = TimeSeriesSplit(n_splits=5)

    for i in range(date_num-p):
        if i >= numOfForwardDays:
            break
        #Prepare training and testing data for the current testing day
        trainingDates=[date_index[i]]
        for j in range(i+1,i+p):
            trainingDates.append(date_index[j])        

        dataPd=data[data.date.isin(trainingDates)]
        data1d=data[data.date==date_index[i+p]]

        #Building models with LightGBM models for each label
        lgbmModels = {}
        LGBMModels = {}
        for j in range(1, 11):
            lgbmModels['Y_M_{}'.format(str(j))] = lgb.LGBMRegressor()
        
        #Use a dictionry to hold training label data
        y_training = {}
        for j in range(1, 11):
            y_training['Y_M_{}'.format(str(j))] = dataPd['Y_M_{}'.format(str(j))]
        
        #Hold feature data for training
        x_training = pd.DataFrame()
        for j in range(len(featureColumns)):
            x_training[featureColumns[j]] = dataPd[featureColumns[j]]

        #Use a dictionry to hold testing label data
        y_testing = {}
        for j in range(1, 11):
            y_testing['Y_M_{}'.format(str(j))] = data1d['Y_M_{}'.format(str(j))]

        #Hold feature data for testing
        x_testing = pd.DataFrame()
        for j in range(len(featureColumns)):
            x_testing[featureColumns[j]] = data1d[featureColumns[j]]

        #Use a dictionry to hold predictions
        y_prediction = {}

        #timing the multi-Process prcessing
        t = time.time()
        #Construction of models
        ##Establish a number of Processs for the calculation tasks
        pool = Pool(processes=numOfProcesses)

        for j in range(1, 11):
            study = optuna.create_study(direction='minimize')
            study.optimize(lambda trial: objective(trial, x_training, y_training['Y_M_{}'.format(str(j))], measure, lookforward_win=j), n_trials=10, n_jobs=-1)
            best_params = study.best_params
            lgbmModels['Y_M_{}'.format(str(j))].set_params(**best_params)
            LGBMModels['Y_M_{}'.format(str(j))] = pool.apply_async(lgbmModels['Y_M_{}'.format(str(j))].fit, args=(x_training, y_training['Y_M_{}'.format(str(j))]))

        pool.close()
        pool.join()
        print(">>>Done with " + str(numOfProcesses) + " processes, and time taken is " + str(time.time() - t)) 
        
        # Save the models after all models are fitted
        # check if dir exists
        if not os.path.exists('./model_params'):
            os.makedirs('./model_params')
        for j in range(1, 11):
            LGBMModels['Y_M_{}'.format(str(j))].get().booster_.save_model('./model_params/model_{}_Y_M_{}.txt'.format(tickers, str(j)))
        
        #Use the models to predict
        for j in range(1, 11):
            y_prediction['Y_M_{}'.format(str(j))] = LGBMModels['Y_M_{}'.format(str(j))].get().predict(x_testing)
        
        #Now, we store modeling results
        for j in range(1, 11):
            oneLineModelResult = []
            oneLineModelResult.extend([str(date_index[i+p])])
            oneLineModelResult.extend(['Y_M_{}'.format(str(j))])
            oneLineModelResult.extend([LGBMModels['Y_M_{}'.format(str(j))].get().score(x_training, y_training['Y_M_{}'.format(str(j))])])
            oneLineModelResult.extend([LGBMModels['Y_M_{}'.format(str(j))].get().score(x_testing, y_testing['Y_M_{}'.format(str(j))])])            
            outputData = pd.concat([outputData, pd.DataFrame(data = [oneLineModelResult], columns = columns1)])

    return outputData

for future_ticker, df in df_all.items():
    numOfPastDays = np.unique(df['date']).size - 1
    numOfForwardDays = 1
    numOfProcesses = 8
    measure = 'cum_ret'

    start_time = time.time()
    outputData = modelConstructionAndForecasting(numOfPastDays, numOfForwardDays, df, numOfProcesses, featureCols, future_ticker, measure)
    end_time = time.time()
    print('Time taken: ', end_time - start_time, 'seconds')
    # Check if dir exists
    if not os.path.exists('./modelStats'):
        os.makedirs('./modelStats')
    outputData.to_csv('./modelStats/modelStats_{}.csv'.format(future_ticker), index=False)

1.8686816455385262


[I 2023-07-30 12:05:02,353] A new study created in memory with name: no-name-66335eaa-0b36-4520-a803-702cd95d2d38
[I 2023-07-30 12:05:07,749] Trial 1 finished with value: -1.830005097883371 and parameters: {'max_depth': 3, 'num_leaves': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.002347692073260415, 'reg_lambda': 0.003341827224461487}. Best is trial 1 with value: -1.830005097883371.


1.836972894239714
0.9953856601434286
1.3845217360124362
2.045231204843789
1.0064225523862458
1.848572304412892
1.9717323277822327
1.9270624916556276
1.932285747729109
0.9053106491566485


[I 2023-07-30 12:05:18,593] Trial 4 finished with value: -1.672383609984498 and parameters: {'max_depth': 7, 'num_leaves': 31, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 1.0565271037988098, 'reg_lambda': 0.014746223615228447}. Best is trial 1 with value: -1.830005097883371.


1.8273718153599456
0.9411653671675655
1.807844338230053


...

1.2862762885095465
1.254059649762516


[I 2023-07-30 14:07:40,313] Trial 8 finished with value: -1.209366258251044 and parameters: {'max_depth': 3, 'num_leaves': 31, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.9464317259332187, 'reg_lambda': 0.26009248110908595}. Best is trial 1 with value: -1.289163884224931.
[I 2023-07-30 14:07:40,313] Trial 2 pruned. Trial was pruned at iteration 213.
[I 2023-07-30 14:07:40,313] Trial 0 pruned. Trial was pruned at iteration 112.
[I 2023-07-30 14:07:40,313] Trial 3 pruned. Trial was pruned at iteration 30.
[I 2023-07-30 14:07:40,313] Trial 4 pruned. Trial was pruned at iteration 74.


1.297016297499972


[I 2023-07-30 14:07:41,619] Trial 9 pruned. Trial was pruned at iteration 0.


>>>Done with 8 processes, and time taken is 609.5509870052338
Time taken:  612.8920819759369 seconds


In [10]:
%pip install optuna

     -------------------------------------- 390.6/390.6 kB 8.1 MB/s eta 0:00:00
     -------------------------------------- 224.5/224.5 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
